In [1]:
import numpy as np
import pandas as pd
import os
import gc
import warnings
import statsmodels.api as sm
import sklearn
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

warnings.filterwarnings('ignore')
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#### Importing DataSets

In [4]:
#Import datasets
train = pd.read_csv("D:/Bootcamp/Project-Sales/Data_set/train.csv")
test = pd.read_csv("D:/Bootcamp/Project-Sales/Data_set//test.csv")
stores = pd.read_csv("D:/Bootcamp/Project-Sales/Data_set//stores.csv")

#### Checking for null values

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [7]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


#### Converting data from object datatype to datetime datatypes

In [12]:
train["date"]=pd.to_datetime(train["date"])
test["date"]=pd.to_datetime(test["date"])


In [13]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


In [14]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [15]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### 2. Data Processing¶
From train.csv and transactions.csv:

date - The date when the data was recorded.

store_nbr - Identifies the store at which the products are sold.

family - Identifies the type of product sold.

sales - Gives the total sales for a product family at a particular store at a given date. Fractional values are possible.

onpromotion - Gives the total number of items in a product family that were being promoted at a store at a given date.

transactions - The total number of transactions that occurred in a store on a given date.

Remove Sales for Stores who do not Sell a Particular Family of Products
After a quick look at our training dataset, we can see that there are a lot of zeroes. It is possible that some stores do not sell certain products as they are not the right store for that product. In that case, we will remove those values as when forecasted, they should not be having any sales.

In [16]:
zeros = train.groupby(['id', 'store_nbr', 'family']).sales.sum().reset_index().sort_values(['family','store_nbr'])
zeros = zeros[zeros.sales == 0]
zeros

,id,store_nbr,family,sales
0,0,1,AUTOMOTIVE,0.0
10692,10692,1,AUTOMOTIVE,0.0
30294,30294,1,AUTOMOTIVE,0.0
40986,40986,1,AUTOMOTIVE,0.0
53460,53460,1,AUTOMOTIVE,0.0
...,...,...,...,...
2981153,2981153,54,SEAFOOD,0.0
2984717,2984717,54,SEAFOOD,0.0
2986499,2986499,54,SEAFOOD,0.0
2993627,2993627,54,SEAFOOD,0.0


In [17]:
#full outer joining the tables and removing the rows where they match to get rid of the zeros
join = train.merge(zeros[zeros.sales == 0].drop("sales",axis = 1), how='outer', indicator=True)
train1 = join[~(join._merge == 'both')].drop(['id', '_merge'], axis = 1).reset_index()
train1 = train1.drop(['index', 'onpromotion'], axis=1)
train1

,date,store_nbr,family,sales
0,2013-01-01,25,BEAUTY,2.000
1,2013-01-01,25,BEVERAGES,810.000
2,2013-01-01,25,BREAD/BAKERY,180.589
3,2013-01-01,25,CLEANING,186.000
4,2013-01-01,25,DAIRY,143.000
...,...,...,...,...
2061753,2017-08-15,9,POULTRY,438.133
2061754,2017-08-15,9,PREPARED FOODS,154.553
2061755,2017-08-15,9,PRODUCE,2419.729
2061756,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000


### Cluster Stores¶
The training data set is what we will use to create our models. There are 54 stores, each with 33 product families. As this is a large dataset with over three million rows, grouping the stores into clusters will greatly reduce the computation of our following models. Fortunately, the stores are already clustered for us in the 'stores' dataset into seventeen clusters.

In [18]:
def group_clusters (df) :
    #left join train and stores on store number
    jointr = df.merge(stores, on='store_nbr', how='left', indicator=False)

    #replacing all store numbers with their cluster and grouping them by cluster
    grouped = jointr.groupby(['date', 'cluster', 'family']).sum('sales').reset_index()

    #removing columns id, store_nbr, and type as they are aggregated values with no significance
    grouped = grouped.drop(['store_nbr'], axis=1)

    return grouped


In [19]:
grouped = group_clusters (train1)
grouped

,date,cluster,family,sales
0,2013-01-01,1,BEAUTY,2.000
1,2013-01-01,1,BEVERAGES,810.000
2,2013-01-01,1,BREAD/BAKERY,180.589
3,2013-01-01,1,CLEANING,186.000
4,2013-01-01,1,DAIRY,143.000
...,...,...,...,...
742573,2017-08-15,17,PLAYERS AND ELECTRONICS,25.000
742574,2017-08-15,17,POULTRY,686.941
742575,2017-08-15,17,PREPARED FOODS,91.976
742576,2017-08-15,17,PRODUCE,5031.190
